In [53]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from urllib.parse import urlparse
import whois
import re
import ssl
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests

## Pregunta 1

In [54]:
data = pd.read_csv('dataset_pishing.csv')

data.head(5)


,url,status
0,http://www.crestonwood.com/router.php,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,phishing
2,https://support-appleld.com.secureupdate.duila...,phishing
3,http://rgipt.ac.in,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,legitimate


In [55]:
data.describe()

,url,status
count,11430,11430
unique,11429,2
top,http://e710z0ear.du.r.appspot.com/c:/users/use...,legitimate
freq,2,5715


In [56]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     11430 non-null  object
 1   status  11430 non-null  object
dtypes: object(2)
memory usage: 178.7+ KB


## Pregunta 2

In [57]:
data['status'].value_counts()

status
legitimate    5715
phishing      5715
Name: count, dtype: int64

El dataset está balanceado porque tiene la misma cantidad de ejemplos para cada clase.

### 1. ¿Qué ventajas tiene el análisis de una URL contra el análisis de otros datos, cómo el tiempo de vida del dominio, o las características de la página Web?

- Rapidez: El análisis de URL es generalmente más rápido que otras técnicas, como el tiempo de vida del dominio, lo que permite una detección más inmediata.

- Información inmediata: La URL proporciona información directa sobre la dirección web, lo que facilita la evaluación inmediata de su validez.

- Facilidad de extracción: Las características de una URL son fáciles de extraer y analizar en comparación con otras métricas, como el tiempo de vida del dominio.

- Accesibilidad: Las URLs son públicas y fácilmente accesibles, permitiendo análisis sin requerir acceso a información privada.

### 2. ¿Qué características de una URL son más prometedoras para la detección de phishing?

- Longitud de la URL: Las URL de phishing a menudo tienden a ser más largas y complejas.

- Presencia de caracteres especiales: La presencia de caracteres inusuales o codificados puede indicar un intento de ocultar la verdadera identidad de la URL.

- Uso de subdominios: Phishing a veces utiliza subdominios para imitar sitios legítimos, por lo que analizar su presencia puede ser útil.

- Coincidencia con marcas conocidas: La presencia de nombres de marcas en la URL puede indicar intentos de suplantación de identidad.

- Verificación de protocolo: La ausencia de HTTPS puede ser una señal de que la URL es sospechosa.

- Fecha de creación: Analizar la fecha de creación de la URL puede ayudar a identificar dominios recientemente registrados, lo cual es común en sitios de phishing.

- Registro de WHOIS: Revisar la información de WHOIS para evaluar la autenticidad del registrante y la información de contacto.

- Redirecciones múltiples: Phishing a menudo utiliza múltiples redirecciones para ocultar la verdadera dirección del sitio.

- Similitud con URLs legítimas: Comparar la URL con la de sitios legítimos para identificar posibles intentos de imitación.

- Análisis de palabras clave: Identificar palabras clave sospechosas o mal escritas en la URL que podrían indicar un intento de engaño.

- Presencia de números de IP en lugar de nombres de dominio: Las URLs que utilizan direcciones IP en lugar de nombres de dominio pueden ser indicativas de intentos de phishing.

- Coincidencia con patrones conocidos de phishing: Utilizar patrones conocidos de URLs de phishing para identificar similitudes.

- Análisis de caducidad del dominio: Evaluar la fecha de caducidad del dominio puede revelar la intención a largo plazo del sitio.

- Evaluación de popularidad del dominio: Analizar la popularidad de un dominio puede ayudar a identificar URLs sospechosas.


In [58]:
#Funciones

def longitud_url(url):
    return len(url)

def caracteres_especiales(url):
    return sum(c.isalnum() == False for c in url)

def uso_subdominios(url):
    return len(urlparse(url).netloc.split('.'))

def coincidencia_marcas(url, marcas_conocidas):
    return any(brand.lower() in url.lower() for brand in marcas_conocidas)

def verificar_https(url):
    return 1 if urlparse(url).scheme == 'https' else 0


def similitud_url_legitimas(url, urls_legitimas_conocidas):
    vectorizer = CountVectorizer().fit_transform([url] + urls_legitimas_conocidas)
    similarity_matrix = cosine_similarity(vectorizer)
    return similarity_matrix[0, 1:]

def palabras_clave_sospechosas(url, palabras_sospechosas):
    return sum(1 for palabra in palabras_sospechosas if palabra.lower() in url.lower())

def ip_en_lugar_de_dominio(url):
    return 1 if re.match(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', urlparse(url).netloc) else 0

def patrones_conocidos_phishing(url, patrones_phishing_conocidos):
    return any(re.search(patron, url) for patron in patrones_phishing_conocidos)


def url_acortada(url):
    servicios_acortadores = ['bit.ly', 'tinyurl.com', 'goo.gl', 'ow.ly', 't.co']
    netloc = urlparse(url).netloc
    return any(servicio in netloc for servicio in servicios_acortadores)
    
def puerto_no_estandar(url):
    puerto = urlparse(url).port
    return 1 if puerto and puerto not in [80, 443] else 0

# Presencia de términos de seguridad falsos en la URL
def terminos_seguridad_falsos(url):
    terminos_falsos = ['secure', 'account', 'verify', 'banking']
    return any(termino in url.lower() for termino in terminos_falsos)

# Profundidad de la URL (número de '/' en el path)
def profundidad_url(url):
    path = urlparse(url).path
    return path.count('/')

# Presencia de archivos descargables sospechosos (ej. .exe, .zip)
def archivo_descargable_sospechoso(url):
    extensiones_sospechosas = ['.exe', '.zip', '.rar', '.dmg']
    return any(url.lower().endswith(ext) for ext in extensiones_sospechosas)



In [59]:
data['longitud_url'] = data['url'].apply(longitud_url)
data['caracteres_especiales'] = data['url'].apply(caracteres_especiales)
data['uso_subdominios'] = data['url'].apply(uso_subdominios)
data['verificar_https'] = data['url'].apply(verificar_https)
data['similitud_url_legitimas'] = data['url'].apply(lambda x: similitud_url_legitimas(x, urls_legitimas_conocidas=['http://legitimo.com']))
data['palabras_clave_sospechosas'] = data['url'].apply(lambda x: palabras_clave_sospechosas(x, palabras_sospechosas=['phishing']))
data['ip_en_lugar_de_dominio'] = data['url'].apply(ip_en_lugar_de_dominio)
data['url_acortada'] = data['url'].apply(url_acortada)
data['puerto_no_estandar'] = data['url'].apply(puerto_no_estandar)
data['terminos_seguridad_falsos'] = data['url'].apply(terminos_seguridad_falsos)
data['profundidad_url'] = data['url'].apply(profundidad_url)
data['archivo_descargable_sospechoso'] = data['url'].apply(archivo_descargable_sospechoso)

In [60]:
label_encoder = LabelEncoder()
data['status_binario'] = label_encoder.fit_transform(data['status'])

data = data.drop(['url'], axis=1)


print("DataFrame después de las modificaciones:")
print(data.head())


DataFrame después de las modificaciones:
       status  longitud_url  caracteres_especiales  uso_subdominios  \
0  legitimate            37                      7                3   
1    phishing            77                      7                2   
2    phishing           126                     19                5   
3  legitimate            18                      5                3   
4  legitimate            55                     10                3   

   verificar_https similitud_url_legitimas  palabras_clave_sospechosas  \
0                0    [0.4714045207910318]                           0   
1                0    [0.4714045207910318]                           0   
2                1    [0.2886751345948129]                           0   
3                0    [0.2886751345948129]                           0   
4                0     [0.408248290463863]                           0   

   ip_en_lugar_de_dominio  url_acortada  puerto_no_estandar  status_binario  
0        

In [63]:
print(data['ip_en_lugar_de_dominio'].value_counts())

ip_en_lugar_de_dominio
0    11333
1       97
Name: count, dtype: int64
